In [14]:
import json
from typing import Annotated, TypedDict, Literal
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.messages import SystemMessage, HumanMessage,BaseMessage, ToolMessage, AIMessage
from langgraph.graph import StateGraph, END, add_messages
from langgraph.prebuilt import ToolNode

In [15]:
model = ChatOpenAI(
    base_url="https://ws-02.wade0426.me/v1",
    api_key="day3hw",
    model="Qwen/Qwen3-VL-8B-Instruct",
    temperature=0.1
)

In [16]:
VIP_LIST = ["AI哥", "一龍馬"]

@tool
def extract_order_data(name: str, phone: str, product: str, quantity: int, address: str):
    """
    資料提取專用工具
    專門用於從非結構話文本中提取訂單相關資訊（姓名，電話，商品，數量，地址）。
    """

    return {
        "name": name,
        "phone": phone,
        "product": product,
        "quantity": quantity,
        "address": address
    }

In [17]:
llm_with_tools = model.bind_tools([extract_order_data])
tool_node = ToolNode([extract_order_data])

class AgentState(TypedDict):
    messages: Annotated[list[BaseMessage], add_messages]

In [18]:
def agent_node(state: AgentState):
    response = llm_with_tools.invoke((state["messages"]))
    return {"messages": [response]}

In [19]:
def human_review_node(state: AgentState):
    print("\n" + "="*30)
    print("觸發人工審核機制：檢測到 VIP 客戶！")
    print("="*30)

    last_msg = state["messages"][-1]
    print(f"待審核資料：{last_msg.content}")

    review = input(">>> 管理員請批示（輸入 'ok' 通過，其他則拒絕）：")

    if review.lower() == "ok":
        return {
            "messages": [
                AIMessage(content="收到訂單資料，因偵測到 VIP 客戶，系統將轉到交人工審核..."),
                HumanMessage(content="[系統公告] 管理員已人工審核通果此  VIP 訂單，請繼續完成後續動作。")
            ]
        }
    else:
        return {
            "messages": [
                AIMessage(content="已收到訂單資料，等待人工審核結果..."),
                HumanMessage(content="[系統公告] 管理拒絕了此訂單，請取消交易並告知用戶。")
            ]
        }

In [20]:
def entry_router(state: AgentState):
    last_msg = state["messages"][-1]
    if last_msg.tool_calls:
        return "tools"
    return END

In [21]:
def post_tool_router(state: AgentState) -> Literal["human_review", "agent"]:
    messages = state["messages"]
    last_msg = messages[-1]

    if isinstance(last_msg, ToolMessage):
        try:
            data = json.loads(last_msg.content)
            user_name = data.get("name", "")

            if user_name in VIP_LIST:
                print(f"Debug: 發現 VIP [{user_name}] -> 轉向人工審核")
                return  "human_review"

        except Exception as e:
            print(f"JSON 解析錯誤: {e}")

    return "agent"

In [22]:
workflow = StateGraph(AgentState)

workflow.add_node("agent", agent_node)
workflow.add_node("tools", tool_node)
workflow.add_node("human_review", human_review_node)

workflow.set_entry_point("agent")

workflow.add_conditional_edges(
    "agent",
    entry_router,
    {"tools": "tools", END: END},
)

workflow.add_conditional_edges(
    "tools",
    post_tool_router,
    {
        "human_review": "human_review",
        "agent": "agent",
    }
)

workflow.add_edge("human_review", "agent")

app = workflow.compile()
print(app.get_graph().draw_ascii())

                  +-----------+              
                  | __start__ |              
                  +-----------+              
                        *                    
                        *                    
                        *                    
                    +-------+                
                    | agent |*               
                 ...+-------+ ***            
              ...       .        ****        
           ...          .            ***     
         ..             .               ***  
+---------+         +-------+              **
| __end__ |         | tools |             ** 
+---------+         +-------+            *   
                             .         **    
                              ..     **      
                                .   *        
                          +--------------+   
                          | human_review |   
                          +--------------+   


In [24]:
if __name__ == "__main__":
    print(f"VIP 名單： {VIP_LIST}")

    while True:
        user_input = input("\nUser: ")
        if user_input.lower() in ["exit", "quit"]: break

        for event in app.stream({"messages": [HumanMessage(content=user_input)]}):
            for key, value in event.items():
                if key == "agent":
                    msg = value["messages"][-1]
                    if not msg.tool_calls:
                        print(f" -> [Agent]: {msg.content}")
                    elif key == "human_review":
                        print(f" -> [Human]: 審核完成")

VIP 名單： ['AI哥', '一龍馬']
Debug: 發現 VIP [一龍馬] -> 轉向人工審核

觸發人工審核機制：檢測到 VIP 客戶！
待審核資料：{"name": "一龍馬", "phone": "unknown", "product": "豪華型電腦", "quantity": 1, "address": "銀河系"}
 -> [Agent]: 好的，豪華型電腦已安排配送至銀河系，預計稍後送達。請留意收貨！


KeyboardInterrupt: Interrupted by user